In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving churnbagging.csv to churnbagging.csv


In [ ]:
data=pd.read_csv("churnbagging.csv")
data


,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,current_balance,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,0,0.0,0,187.0,2,755,224.0,1458.71,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,6,2531,42,0,2.0,0,1494.0,3,388,58.0,927.72,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1
2,7,263,42,1,0.0,0,1096.0,2,1666,60.0,15202.20,16059.34,15211.29,13798.82,0.36,0.36,857.50,286.07,15719.44,15349.75,0
3,8,5922,72,0,0.0,1,1020.0,1,1,98.0,7006.93,7714.19,7859.74,11232.37,0.64,0.64,1299.64,439.26,7076.06,7755.98,0
4,9,1145,46,0,0.0,0,623.0,2,317,172.0,10096.58,8519.53,6511.82,16314.17,0.27,0.27,443.13,5688.44,8563.84,5317.04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22062,30295,2041,42,0,0.0,0,146.0,2,286,57.0,7493.69,6777.41,8082.48,3515.03,0.51,90.10,1103.20,1183.04,7956.03,7431.36,0
22063,30297,1845,10,1,0.0,3,1020.0,2,1207,70.0,1076.43,1076.43,2282.19,2787.70,0.30,0.30,0.30,0.30,1076.43,1076.43,0
22064,30298,4919,34,1,0.0,0,1046.0,2,223,14.0,3844.10,4069.21,3668.83,3865.55,1.71,2.29,901.00,1014.07,3738.54,3690.32,0
22065,30299,297,47,0,0.0,2,1096.0,2,588,0.0,65511.97,61017.55,53444.81,21925.81,4666.84,3883.06,168.23,71.80,61078.50,57564.24,1


In [ ]:
predictors = data.drop(columns=['churn','customer_id'])
target = data['churn']

In [ ]:
#splitting data
train = data.sample(frac=0.75, replace = False)
test = data.append(train)
test = test.drop_duplicates(keep=False)

#check
train.shape, test.shape

((16550, 21), (5517, 21))

In [ ]:
# step1 : Setting n_trees_trees
n_trees = 100

In [ ]:
# step2 : Making n_trees bootstrap samples (hint: search pd.sample() function)

def BootStrap(data, n_samples, fraction = 1):
    '''
    function to generate boot strap samples.
    '''
    bootstraps = np.zeros(shape = (int(data.shape[0]*fraction), data.shape[1], n_samples))

    for i in range(n_samples):
        bootstraps[:,:,i] = data.sample(frac=fraction, replace=True).values

    return bootstraps

In [ ]:
# Testing function (do not change)
print('observations, columns, samples')
print(BootStrap(train, n_trees, fraction=1).shape)
print(BootStrap(train, n_trees, fraction=0.75).shape)
print(BootStrap(train, n_trees, fraction=0.5).shape)

observations, columns, samples
(16550, 21, 100)
(12412, 21, 100)
(8275, 21, 100)


In [ ]:
# Step3.1 : create a function called generate predictions which will train a decision tree model over one bootstrap sample and return its predictions on test set

def generate_predictions(train_x, train_y, test_x):
    '''
    train_x: independent variables of sample
    train_y: target variable of sample
    test_x: independent variables of test data

    Return: prediction for the test_x
    '''
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(max_depth=7, class_weight='balanced')
    classifier.fit(train_x, train_y)
    predictions = classifier.predict(test_x)
    return predictions


In [ ]:
## testing function
print('Expected length of predictions:', test.shape[0])
print("generated predictions shape: ",generate_predictions(train.drop(columns=['churn']), train['churn'], test.drop(columns=['churn'])).size)
# output numbers should be same

Expected length of predictions: 5517
generated predictions shape:  5517


In [ ]:
#Step3.2 : Create a function called Bagging which uses the generate_prediction and bootstrap functions to generate predictions for all the bootstrap samples and calculate mode of predictions for eachhh sample.

def Bagging(n_trees, train_data, test_data, sample_fraction):
    '''
    N-trees: number of trees
    Return: final predictions of overall bagging technique
    '''
    bootstrap_samples = BootStrap(train_data, n_samples = n_trees, fraction = sample_fraction)
    multi_predictions = np.zeros(shape = (int(test_data.shape[0]),n_trees))

    for i in range(n_trees):
        tmp = generate_predictions(bootstrap_samples[:,:-1,i],bootstrap_samples[:,-1,i], test_data.drop(columns=['churn']))
        multi_predictions[:,i] = tmp

    from scipy.stats import mode
    final_prediction = mode(multi_predictions,axis=1)
    return final_prediction[0]

In [ ]:
(Bagging(n_trees, train, test, 1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has featu

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]])